In [1]:
from field import F, P, Z

# Generator
# G = F(85408008396924667383611388730472331217)
G = F(3)

for i in range(0, P):
    print(i, G ** i)

0 1
1 3
2 9
3 10
4 13
5 5
6 15
7 11
8 16
9 14
10 8
11 7
12 4
13 12
14 2
15 6
16 1


In [1]:
from polynomial import Polynomial

p = Polynomial([-1, 0, 1])
q = Polynomial([1,  1])

(p + q)(1)

2

In [6]:
import merkle

ls = ["A", "B", "C", "D", "E"]
hs = [merkle.hash_leaf(l) for l in ls]

for h in hs:
    print(h)

root = merkle.commit(hs)
print("root", root)

i = 2
proof = merkle.open(hs, i)
merkle.verify(proof, root, hs[i], i)

559aead08264d5795d3909718cdd05abd49572e84fe55590eef31a88a08fdffd
df7e70e5021544f4834bbee64a9e3789febc4be81470df629cad6ddb03320a5c
6b23c0d5f35d1b11f9b683f0b0a617355deb11277d91ae091d399c655b87940d
3f39d5c348e5b79d06e842c114e6cc571583bbf44e4b0ebfda1a01ec05745d43
a9f51566bd6705f7ea6ad54bb9deb449f795582d6529a0e22207b8981233ec58
root 2db1790243fe117685d21ed0ff5005d9832e5f32bf5b2b02cddf0f07a34421b2


True

In [30]:
# FRI
# message (M) -> poly degree < M - 1 -> RS code (N) <= |L| size of evaluation domain
# N = |L| = 2**K
# L = [1, w, w^2, ..., w^(N - 1)], w is Nth root of unity
# prime field F_p, |F_p| > |L|, N divides p - 1 (needed for finding primitive root of unity)

# Find L
n = 16
p = 337
# Prime divisors of 337 - 1 = 336
divisors = [2, 3, 7]
# Find generators
gs = []
for x in range(1, p):
    if all(pow(x, (p - 1) // q, p) != 1 for q in divisors):
        gs.append(x)

# Check order of the group generated by g (should be p - 1, multiplicative group excluding 0)
g = gs[0]
group = set()
for i in range(0, p - 1):
    x = pow(g, i, p)
    if x in group:
        print(f'{x} = {g}^{i} is already included')
    group.add(x)
print(f'Order of multiplicative group generated by g = {g}:', len(group))

# Primitive Nth root
r = pow(g, (p - 1) // n, p)
print(f'Primitive {n} th root:', r)
# Check r is a primitive Nth root
for i in range(1, 16):
    x = pow(r, i, p)
    # print(f'{r}^{i} = {x}')
    assert x != 1
assert pow(r, 16, p) == 1

# Evaluation domain L
L = [pow(r, i, p) for i in range(0, n)]
L

Order of multiplicative group generated by g = 10: 336
Primitive 16 th root: 191
191^1 = 191
191^2 = 85
191^3 = 59
191^4 = 148
191^5 = 297
191^6 = 111
191^7 = 307
191^8 = 336
191^9 = 146
191^10 = 252
191^11 = 278
191^12 = 189
191^13 = 40
191^14 = 226
191^15 = 30


[1, 191, 85, 59, 148, 297, 111, 307, 336, 146, 252, 278, 189, 40, 226, 30]

In [ ]:
# Polynomials

# 0 <= P(x) <= 9 integer 
# for all 1 <= x <= 1,000,000

# C(x) = x*(x-1)*...*(x-9)
# C(x) = 0 if 0 <= x <= 9
def c(x):
    v = 1
    for i in range(10):
        v *= (x - i)
    return v

# Z(x) = (x-1)*(x-2)*...*(x-1e6)
def z(x, n):
    v = 1
    for i in range(1, n + 1):
        v *= (x - i)
    return v

# C(P(x)) = 0 for 1 <= x <= 1,000,000
# C(P(x)) = Z(x)D(x)

# Polynomial commitment
# 1. Merkle tree of P(x) and D(x) at 1,000,000,000 points
# 2. Verifier randomly selects 16 values between 1 and 1,000,000,000
#    and asks the prover to provide the Merkle branches of P(x) and D(x)
# 3. Verifier checks
#    - Merkle proofs
#    - C(P(x)) = Z(x)D(x), P(x) and D(x) are provided in the Merkle proof

# Bivariate polynomial
# f(x) = polynomial with degree < 1,000,000
# Find g(x,y) such that g(x, x^1000) = f(x)

# 1. Polynomial commintment on g(x, y)
# for {(x, y^1000) for 1 <= x <= N and 1 <= y <= N}
# N = 1,000,000,000

# 2. Verifier picks randomly picks a few rows and columns
#    and for each row and column, asks for a few sample of points
#    one point in the sample is on the diagnol (x, x^1000)

# 3. Prover replies with does points and Merkle proofs

# TODO: why use bivariate polynomials?
# 4. - Verifier checks Merkle proofs
#    - Check polynomial corresponds to a low degree polynomial.
#      Degree less than number of samples requested.